## Gauß Algorithmus in Matrixform, auch: LR-Zerlegung

### Dezimalzahlen in Python

Python module: decimal (Dezimalzahlen)<br>
https://docs.python.org/3/library/decimal.html

Einstellungen

In [1]:
import math
import copy
from decimal import *
from __future__ import annotations

# Einstellungen
getcontext()

Context(prec=28, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])

In [2]:
# Stellenzahl
#
getcontext().prec = 2
#getcontext().prec = 3

# Rundungsregel: 
# ROUND_FLOOR (nach -inf), ROUND_CEILING (nach +inf), 
# ROUND_DOWN (nach 0), ROUND_UP (weg von 0),   
# ROUND_HALF_EVEN, ROUND_HALF_UP (verwenden wir in der Vorlesung)
getcontext().rounding = ROUND_HALF_UP

# The significance of a new Decimal is determined solely by the number 
# of digits input. Context precision and rounding only come into play 
# during arithmetic operations.
print(Decimal("1.25"))
print("jetzt gerundet:", Decimal("1.25")+Decimal("0"))
print(Decimal("-1.25"))
print("jetzt gerundet:", Decimal("-1.25")+Decimal("0"))

1.25
jetzt gerundet: 1.3
-1.25
jetzt gerundet: -1.3


class Matrix verwendet decimal

In [3]:
class Matrix:
    def __init__(self, n : int, m : int):
        ''' Null-Matrix mit Format (n, m) '''
        self.n = n
        self.m = m
        self.makeZero()
        
    def makeZero (self) -> None:
        ''' Null-Matrix '''
        self.r = [[Decimal("0") for j in range(self.m)]  for i in range(self.n)]
    def makeUnit (self) -> None:
        ''' Einheits-Matrix '''
        self.r = [[Decimal("0") for j in range(self.m)]  for i in range(self.n)]
        for i in range(self.n):
            self.r[i][i] = Decimal("1")

    def add (self, b : Matrix) -> Matrix:
        ''' Matrixaddition '''
        assert(self.n == b.n and self.m == b.m)
        result = Matrix(self.n, self.m)
        for i in range(self.n):
            for j in range(self.m):
                result.r[i][j] += b.r[i][j]
        return result
    def mult (self, b : Matrix) -> Matrix:
        ''' Matrixmultiplikation '''
        assert(self.m == b.n)
        result = Matrix(self.n, b.m)
        for i in range(self.n):
            for j in range(b.m):
                for k in range(self.m):
                    #print(self.r[i][k] * b.r[k][j])
                    result.r[i][j] += self.r[i][k] * b.r[k][j]
        return result
    def permMult (self, b : Matrix, Pr : list[int]) -> Matrix:
        ''' Matrixaddition, 1.Matrix permutiert. '''
        assert(self.m == b.n)
        result = Matrix(self.n, b.m)
        for i in range(self.n):
            ip = Pr[i]
            for j in range(b.m):
                for k in range(self.m):
                    result.r[i][j] += self.r[ip][k] * b.r[k][j]
        return result
    def multPerm (self, b : Matrix, Pr : list[int]) -> Matrix:
        ''' Matrixaddition, 2.Matrix permutiert. '''
        assert(self.m == b.n)
        result = Matrix(self.n, b.m)
        for i in range(self.n):
            for j in range(b.m):
                for k in range(self.m):
                    kp = Pr[k]
                    result.r[i][j] += self.r[i][k] * b.r[kp][j]
        return result
    def multElem (self, sk : Decimal) -> Matrix:
        ''' Matrixskalierung: (sk, .., sk)*self '''
        result = Matrix(self.n, selm)
        for i in range(self.n):
            for j in range(self.m):
                result.r[i][j] *= sk
        return result
    def multRow (self, diag : list[Decimal]) -> Matrix:
        ''' Matrixskalierung, zeilenweise: diag*self '''
        assert(self.n == len(diag))
        result = copy.deepcopy(self)
        for i in range(self.n):
            for j in range(self.m):
                result.r[i][j] *= diag[i]
        return result
    def multCol (self, diag : list[Decimal]) -> Matrix:
        ''' Matrixskalierung, spaltenweise: self*diag '''
        assert(self.m == len(diag))
        result = copy.deepcopy(self)
        for i in range(self.n):
            for j in range(self.m):
                result.r[i][j] *= diag[j]
        return result
    def row (self, r : int) -> Matrix:
        ''' Matrixzeile r als neue Zeilenmatrix. '''
        assert(0 <= r and r < self.n)
        result = Matrix(1, self.m)
        for j in range(self.m):
            result.r[0][j] = copy.deepcopy(self.r[r])
        return result
    def col (self, c : int) -> Matrix:
        ''' Matrixspalte c als neue Spaltenmatrix. '''
        assert(0 <= c and c < self.m)
        result = Matrix(self.n, 1)
        for i in range(self.n):
            result.r[i][0] = self.r[i][c]
        return result

    def __repr__(self) -> str:
        ''' '''
        return str(self.r)
    def __str__(self) -> str:
        ''' '''
        return "Matrix(" + str(self.n) + "," + str(self.m) + ")" + repr(self)


In [4]:
def LUDecomp (A : Matrix) -> tuple[Matrix,Matrix,list[int]]:
    ''' LUDecomp mit Teilpivotisierung. '''
    assert(A.n == A.m)
    L  = [Matrix(A.n, A.n) for k in range(A.m-1)]
    Pr = [k for k in range(A.n)]
    R  = copy.deepcopy(A)
    
    # A.m-1 Schritte
    for step in range(A.m-1):
        # Spalte "step"
        # suche pivot element "ip"
        ip = Pr[step]
        for i in range(step, A.n):
            #print(step, R.r[i][step], R.r[ip][step])
            if abs(R.r[Pr[i]][step]) > abs(R.r[ip][step]):
                ip = Pr[i]
        #ip = step # keine Pivotisierung, nur Test!
        Pr[step] = ip
        Pr[ip]   = step
        print("Schritt ", step, ":")
        print("Pivotzeile=", ip, ", Pivotelement=", R.r[ip][step])
            
        # berechne L_step
        L[step].makeUnit()
        if R.r[Pr[step]][step] == Decimal("0"):
            print("Spalte ist Null, keine Zeilenoperation!")
            continue
        # Einheitsmatrix, dann Spalte step unten geändert
        for i in range(step+1, A.n):
            # Faktor fuer Zeile i
            q = R.r[Pr[i]][step]/R.r[Pr[step]][step]
            L[step].r[i][step] = -q

            # Zeilenoperation: A.Zeile i -q*Pivotzeile (A.Zeile step)
            # gespeichert in R.Zeile i
            # Spalte j=step ist 0
            R.r[Pr[i]][step] = Decimal("0")
            # Spalte j berechnen
            for j in range(step+1, A.m):
                R.r[Pr[i]][j] -= q*R.r[Pr[step]][j]
        # Matrix R nach Schritt step
        print("R_%i" % (step), "=", R)
        
    print("Zeilenpermutation bzgl. A oder R: ", Pr)
    return (L, R, Pr)

In [5]:
def LUSolve (L : list[Matrix], R : Matrix, Pr : list[int], b : Matrix) -> Matrix:
    ''' LUSolve mit LR-Zerlegung und rechter Seite b. '''
    assert(L[0].n == L[0].m)
    assert(R.n == R.m)
    assert(b.n == L[0].m and b.m == 1)
    assert(b.n == R.n and b.m == 1)
    assert(len(Pr) == R.n)
    
    # Loesungsvektor
    x = Matrix(R.m, 1)

    # berechne Spaltenvektor (L(n-1)*..*(L1*b))
    # effizienter, wenn in eine Matrix L zusammengefuegt!
    #print(b)
    for (k, Lk) in enumerate(L):
        if k == 0: # permutiere b in der ersten Mult
            bL = Lk.multPerm(b, Pr)
        else:
            bL = Lk.mult(b)
        b  = bL
    print(k, bL)
    
    # Rueckwaertseinsetzen: loese R*x = bL
    for i in range(R.n-1, -1, -1):
        x.r[i][0] = bL.r[i][0] 
        for j in range(i+1, R.m):
            x.r[i][0] -= R.r[Pr[i]][j]*x.r[j][0]
        if R.r[Pr[i]][i] == Decimal("0"):
            if bL.r[i][0] != Decimal("0"):
                print("keine Lösung!")
                return (x)
            #else:
            continue
        x.r[i][0] /= R.r[Pr[i]][i]
        
    return (x)

### Nur Teilpivotisierung

In [6]:
# Beispiel aus der Uebung
A = Matrix(2, 2)
A.r[0] = [Decimal("2"), Decimal("400")]
A.r[1] = [Decimal("1"), Decimal("1")  ]
#print(A)
# rechte Seite
b = Matrix(2, 1)
b.r = [[Decimal("200")], 
       [Decimal("1")]]
#print(b)

# LR Zerlegung (implizite Pivotisierung)
(L, R, Pr) = LUDecomp(A)

# Loesung x
(x) = LUSolve(L, R, Pr, b)
x

Schritt  0 :
Pivotzeile= 0 , Pivotelement= 2
R_0 = Matrix(2,2)[[Decimal('2'), Decimal('400')], [Decimal('0'), Decimal('-2.0E+2')]]
Zeilenpermutation bzgl. A oder R:  [0, 1]
0 Matrix(2,1)[[Decimal('2.0E+2')], [Decimal('-99')]]


[[Decimal('0E+1')], [Decimal('0.50')]]

### Äquilibrierung dann Teilpivotisierung

In [7]:
# Beispiel aus der Uebung
sk = [ Decimal("0") for _ in range(A.n) ] 
for i in range(A.n):
    for j in range(A.m):
        sk[i] += abs(A.r[i][j])
sk = [Decimal("1")/sk[i] for i in range(A.n)]

Aq = A.multRow(sk)
#print(Aq)
# rechte Seite
bq = b.multRow(sk)
#print(bq)

# LR Zerlegung (implizite Pivotisierung)
(L, R, Pr) = LUDecomp(Aq)
print("L=", L)
print("R=", R)
print("Pr=", Pr)

# Loesung x
(x) = LUSolve(L, R, Pr, bq)
x

Schritt  0 :
Pivotzeile= 1 , Pivotelement= 0.5
R_0 = Matrix(2,2)[[Decimal('0'), Decimal('1.0')], [Decimal('0.5'), Decimal('0.5')]]
Zeilenpermutation bzgl. A oder R:  [1, 0]
L= [[[Decimal('1'), Decimal('0')], [Decimal('-0.010'), Decimal('1')]]]
R= Matrix(2,2)[[Decimal('0'), Decimal('1.0')], [Decimal('0.5'), Decimal('0.5')]]
Pr= [1, 0]
0 Matrix(2,1)[[Decimal('0.50')], [Decimal('0.50')]]


[[Decimal('0.5')], [Decimal('0.5')]]